# Exercise

In [1]:
# Coap message sizes (in bits)
Lgetreq = 60 * 8
Lgetresp = 55 * 8

# MQTT message sizes (in bits)
Lcon = 54 * 8
Lconack = 47 * 8
Lsub = 58 * 8
Lsuback = 52 * 8
Lpub = 68 * 8

# Energy costs (in millijoule or millijoule/bit)
Etx = 50 * 1e-6
Erx = 58 * 1e-6
Ec = 2.4

EQ1) Compute the total energy consumed by the two battery-powered devices over a period of 24 hours in both cases when using COAP (a) and MQTT (b), using each in its most efficient configuration energy-wise.

<img src="./images/COAP.png" alt="COAP" width="400"/>

<img src="./images/MQTT.png" alt="MQTT" width="400"/>

In [2]:
# Coap
Evalvecoap = Etx*Lgetreq + 48 * (6 * Erx*Lgetresp + Ec)
Etempcoap = Erx*Lgetreq + 48 * 6 * Etx*Lgetresp
Ecoap = Evalvecoap + Etempcoap

print(f"{Ecoap:.2f} mJ")

# MQTT
Evalvemqtt = Etx*Lcon + Erx*Lconack + Etx*Lsub + Erx*Lsuback + 48 * (6 * Erx*Lpub + Ec)
Etempmqtt = Etx*Lcon + Erx*Lconack + 48 * 6 * Etx*Lpub
Emqtt = Evalvemqtt + Etempmqtt

print(f"{Emqtt:.2f} mJ")

128.94 mJ
132.25 mJ


EQ2) Propose atleast one solution for decreasing the energy consumption when passing using the Raspberry PI as a broker. Give a rough estimate of the energy saving that could be obtained with your solution: recompute the energy under your proposed configuration.

<img src="./images/BetterMQTT.png" alt="BetterMQTT" width="400"/>

In [3]:
# Do computation on the Raspberry Pi and share only the average every 30 min with the valve
Evalvemqtt = Etx*Lcon + Erx*Lconack + Etx*Lsub + Erx*Lsuback + 48 * Erx*Lpub
Etempmqtt = Etx*Lcon + Erx*Lconack + 48 * 6 * Etx*Lpub
Emqtt = Evalvemqtt + Etempmqtt

print(f"{Emqtt:.2f} mJ")

9.48 mJ


In [4]:
# MQTTSN
Lreg = 59 * 8 # HP from the exercise session (other dimensions seems similar)
Lregack = 51 * 8 # HP from the exercise session (other dimensions seems similar)
Lpub = Lpub - 8 * 8 # Topic is 10, but TopicID is 2 so 8 less

Evalvemqtt = Etx*Lcon + Erx*Lconack + Etx*Lsub + Erx*Lsuback + 48 * (6 * Erx*Lpub + Ec)
Etempmqtt = Etx*Lcon + Erx*Lconack + Etx*Lreg + Erx*Lregack + 48 * 6 * Etx*Lpub
Emqtt = Evalvemqtt + Etempmqtt

print(f"{Emqtt:.2f} mJ")

130.31 mJ


In [5]:
# MQTTSN + Half Payload (looking online 4 bytes are enough to express a temperature with sufficient precision)
Lreg = 59 * 8
Lregack = 51 * 8
Lpub = Lpub - 4 * 8

Evalvemqtt = Etx*Lcon + Erx*Lconack + Etx*Lsub + Erx*Lsuback + 48 * (6 * Erx*Lpub + Ec)
Etempmqtt = Etx*Lcon + Erx*Lconack + Etx*Lreg + Erx*Lregack + 48 * 6 * Etx*Lpub
Emqtt = Evalvemqtt + Etempmqtt

print(f"{Emqtt:.2f} mJ")

129.32 mJ
